In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")

In [ ]:
data.head()

In [ ]:
data.drop(["source"],axis=1,inplace=True)

In [ ]:
data[310:320]

In [ ]:
import re

In [ ]:
pattern="[a-zA-Z]"
b=[]
for i in range(15000):
    result=re.findall(pattern,data["english_sentence"].iloc[i])
    if len(result)==0:
        b.append(i)
    if len(data["english_sentence"].iloc[i])>100:
        b.append(i)

In [ ]:
for i in range(15000):
    result=re.findall(pattern,data["hindi_sentence"].iloc[i])
    if len(result)!=0:
        b.append(i)
    if len(data["hindi_sentence"].iloc[i])>100:
        b.append(i)
        

In [ ]:
len(b)

In [ ]:
data["hindi_sentence"].iloc[73]

In [ ]:
len(b)

In [ ]:
english_texts=[]
hindi_texts=[]
english_character=[]
hindi_character=[]

In [ ]:
for i in range(15000):
    if i not in b:
        english_texts.append(data["english_sentence"].iloc[i])

In [ ]:
for i in range(15000):
    if i not in b:
         hindi="\t"+data["hindi_sentence"].iloc[i]+"\n"
         hindi_texts.append(hindi)

In [ ]:
len(hindi_texts)

In [ ]:
for i in english_texts:
    for c in i:
        if c not in english_character:
            english_character.append(c)
            english_character.sort()

In [ ]:
for j in hindi_texts:
    for c in j:
        if c not in hindi_character:
            hindi_character.append(c)
            hindi_character.sort()

In [ ]:
english_d={}
for i in range(len(english_character)):
    english_d[english_character[i]]=i

In [ ]:
hindi_d={}
for i in range(len(hindi_character)):
    hindi_d[hindi_character[i]]=i

In [ ]:
english_encoder_tokens = len(english_character)
hindi_decoder_tokens = len(hindi_character)

In [ ]:
max_encoder_seq_length=0
for i in english_texts:
    if len(i)>max_encoder_seq_length:
        max_encoder_seq_length=len(i)

In [ ]:
max_decoder_seq_length=0
for i in hindi_texts:
    if len(i)>max_decoder_seq_length:
        max_decoder_seq_length=len(i)

In [ ]:
max_decoder_seq_length

In [ ]:
encoder_input_data=[]
for bb in range(len(english_texts)):
    a=[]
    b=[]
    c=[]
    k=len(english_texts[bb])
    m=0
    while m<k:
        for char in english_texts[bb][m]:
                           for i in range(len(english_character)):
                                            if english_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    encoder_input_data.append(c)


In [ ]:
encoder_input_data=np.array(encoder_input_data)

In [ ]:
encoder_input_data.shape

In [ ]:
decoder_input_data=[]
for bb in range(len(hindi_texts)):
    a=[]
    b=[]
    c=[]
    k=len(hindi_texts[bb])
    m=0
    while m<k:
        for char in hindi_texts[bb][m]:
                           for i in range(len(hindi_character)):
                                            if hindi_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_decoder_seq_length:
        for i in range(len(hindi_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_input_data.append(c)

In [ ]:
decoder_input_data=np.array(decoder_input_data)

In [ ]:
decoder_input_data.shape

In [ ]:
decoder_target_data=[]
for bb in range(len(hindi_texts)):
    a=[]
    b=[]
    c=[]
    k=len(hindi_texts[bb])
    m=1
    while m<k:
        for char in hindi_texts[bb][m]:
                           for i in range(len(hindi_character)):
                                            if hindi_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    m=m-1
    while m<max_decoder_seq_length:
        for i in range(len(hindi_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_target_data.append(c)

In [ ]:
decoder_target_data=np.array(decoder_target_data)

In [ ]:
decoder_target_data.shape

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 

In [ ]:
from keras.layers import Dense
from keras.layers import LSTM,Input
from keras.models import Model

In [ ]:
encoder_inputs = Input(shape=(None,len(english_character)))
encoder = LSTM(latent_dim,dropout=0.2,return_sequences=True,return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder(encoder_inputs)
encoder = LSTM(latent_dim,dropout=0.2,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_outputs_1)
encoder_states = [state_h_1,state_c_1,state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, len(hindi_character)))
decoder_lstm = LSTM(latent_dim,return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs_1, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h_1,state_c_1])
decoder_lstm_1 = LSTM(latent_dim, return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs, _, _ = decoder_lstm_1(decoder_outputs_1, initial_state=[state_h,state_c])
decoder_dense = Dense(len(hindi_character), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.summary()

In [ ]:
model.layers[2]

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

In [ ]:
model.save("engtohindi.h5")

In [ ]:
import keras

In [ ]:
model = keras.models.load_model("engtohindi.h5")

In [ ]:
encoder_inputs = model.input[0]
encoder_outputs_1, state_h_enc_1, state_c_enc_1 = model.layers[2].output 
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output 
encoder_states = [state_h_enc_1, state_c_enc_1,state_h_enc, state_c_enc]
encoder_model_1 = keras.Model(encoder_inputs, encoder_states)

In [ ]:
decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_state_input_h1 = Input(shape=(latent_dim,),name="input_5")
decoder_state_input_c1 = Input(shape=(latent_dim,),name="input_6")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c,decoder_state_input_h1,decoder_state_input_c1]
decoder_lstm = model.layers[3]
dec_o, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
decoder_lstm_1=model.layers[5]
dec_o_1, state_h1, state_c1 = decoder_lstm_1(
    dec_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h,state_c,state_h1,state_c1]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(dec_o_1)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [ ]:
reverse_input_char_index ={}
for i in range(len(english_character)):
    reverse_input_char_index[i]=english_character[i]
reverse_target_char_index ={}
for i in range(len(hindi_character)):
    reverse_target_char_index[i]=hindi_character[i]

In [ ]:
def decode_sequence(input_seq):
    states_value=encoder_model_1.predict(input_seq)
    target_seq = np.zeros((1, 1, len(hindi_character)))
    target_seq[0, 0, hindi_d["\t"]] = 1.0
    flag=0
    sent=""
    while flag==0:
        output_tokens, h, c,h1,c1 = decoder_model.predict([target_seq] + states_value)
        sample = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sample]
        sent+=sampled_char
        if sampled_char == "\n" or len(sent) > max_decoder_seq_length:
            flag=1
        target_seq = np.zeros((1, 1, len(hindi_character)))
        target_seq[0, 0,sample] = 1.0
        states_value = [h, c,h1,c1]
    return sent

In [ ]:
english_texts[21]

In [ ]:
english='Can you imagine saying that?'

In [ ]:
k=len(english)
m=0
a=[]
b=[]
c=[]
inpu=[]
while m<k:
    for char in english[m]:
        for i in range(len(english_character)):
            if english_d[char]==i:
                a.append(1)
            else:
                a.append(0)
    for kp in a:
        b.append(kp)
    c.append(b)
    b=[]
    a=[]
    m=m+1
while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
inpu.append(c)

In [ ]:
inpu=np.array(inpu)

In [ ]:
d=decode_sequence(inpu)

In [ ]:
print(d)